# Text-guided Inpainting on IPUs using Stable Diffusion - Inference

This notebook demonstrates how a stable diffusion inference pipeline can be run on Graphcore IPUs to replace objects inside an image.

![Inpainting with Stable diffusion](sample_images/inpainting.png)

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------|--------------|
| Image processing | Inpainting | Stable Diffusion (stable-diffusion-v1-5) | N/A | Inference | recommended: 4 | 14 min |


[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

## Environment setup

The best way to run this demo is on Paperspace Gradient's cloud IPUs because everything is already set up for you.

[![Run on Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://ipu.dev/3W56b5R)

To run the demo using other IPU hardware, you need to have the Poplar SDK enabled. Refer to the [Getting Started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for details on how to enable the Poplar SDK. Also refer to the [Jupyter Quick Start guide](https://docs.graphcore.ai/projects/jupyter-notebook-quick-start/en/latest/index.html) for how to set up Jupyter to be able to run this notebook on a remote IPU machine.

In order to improve usability and support for future users, Graphcore would like to collect information about the
applications and code being run in this notebook. The following information will be anonymised before being sent to Graphcore:

- User progression through the notebook
- Notebook details: number of cells, code being run and the output of the cells
- Environment details

You can disable logging at any time by running `%unload_ext graphcore_cloud_tools.notebook_logging.gc_logger` from any cell.

## Dependencies and configuration

Install the dependencies for this notebook.

In [ ]:
!pip install -r requirements.txt
!pip install "ipywidgets>=7,<8"
%load_ext graphcore_cloud_tools.notebook_logging.gc_logger

Values for the virtual IPU Pod size and the cache directories can be configured through environment variables or directly in the notebook:

In [ ]:
import os

n_ipu = int(os.getenv("NUM_AVAILABLE_IPU", 4))
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/") + "/stablediffusion_inpainting"

To download the pre-trained Stable-Diffusion-v1-5 checkpoint, we must first authenticate to the 🤗 Hub. 

1. Create a read access token on the [Hugging Face website](https://huggingface.co/settings/tokens). [Sign up to 🤗](https://huggingface.co/join) if you haven't already.
2. Execute the following cell and input your username and read token.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

If you have not done so already, you will need to accept the User License on the [model page](https://huggingface.co/runwayml/stable-diffusion-inpainting).

## Pipeline creation

We are now ready to import and run the pipeline.

In [ ]:
import torch

from optimum.graphcore.diffusers import IPUStableDiffusionInpaintPipeline

In [ ]:
pipe = IPUStableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting", 
    revision="fp16", 
    torch_dtype=torch.float16,
    n_ipu=n_ipu,
    num_prompts=1,
    num_images_per_prompt=1,
    common_ipu_config_kwargs={"executable_cache_dir": executable_cache_dir}
)

We run a dummy generation step to trigger the one-time compilation process. This should take on the order of 15 minutes.

In [ ]:
image_width = os.getenv("STABLE_DIFFUSION_INPAINT_DEFAULT_WIDTH", default=512)
image_height = os.getenv("STABLE_DIFFUSION_INPAINT_DEFAULT_HEIGHT", default=512)
image_dimensions = (image_width, image_height)

In [ ]:
from PIL import Image

pipe("apple", 
     image=Image.new("RGB", image_dimensions), 
     mask_image=Image.new("RGB", image_dimensions, (255, 255, 255)), 
     guidance_scale=7.5
);

## Image generation

We preprocess and visualize a context image which will be used to initialize the latents passed to the UNet.

In [ ]:
import requests
from io import BytesIO

img_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"
mask_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo_mask.png"

def download_image(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content)).convert("RGB")

In [ ]:
image = download_image(img_url).resize(image_dimensions)
image

In [ ]:
mask_image = download_image(mask_url).resize(image_dimensions)
mask_image

Below you will find an example prompt. We encourage you to try your own!

In [ ]:
prompt = "a mecha robot sitting on a bench"
out = pipe(prompt, image=image, mask_image=mask_image, guidance_scale=7.5)
out.images[0]

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
fig, axs = plt.subplots(1, 3)
fig.set_size_inches(12, 5)
axs[0].imshow(image)
axs[0].set_title("Starting image")
axs[1].imshow(mask_image)
axs[1].set_title("Mask")
axs[2].imshow(out.images[0])
axs[2].set_title("Generated image")
for ax in axs:
    ax.axis("off")
fig.tight_layout()
fig.savefig("sample_images/inpainting.png", dpi=150)

## Release IPUs in use

The IPython kernel has a lock on the IPUs used in running the model, preventing other users from using them. For example, if you wish to use other notebooks after running this notebook, it may be necessary to manually run the cell below to release the IPUs you have been using. This will happen by default if you use the `Run All` notebook option. More information can be found in the notebook on "managing IPU resources" - `useful-tips/managing_ipu_resources.ipynb`.

In [ ]:
pipe.detach_from_device()

## Next steps

Try out our other Stable Diffusion notebooks:

* Text to image generation (Stable Diffusion version 1.5) - `text_to_image.ipynb`
* Text to image generation (Stable Diffusion version 2.0) - `text_to_image2.ipynb`
* Image to image generation - `image_to_image.ipynb`

Also, Try out the other [IPU-powered Jupyter Notebooks](https://www.graphcore.ai/ipu-jupyter-notebooks) to see how how IPUs perform on other tasks.
